In [1]:
from langgraph.graph import StateGraph, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode
import os
import json

In [2]:
# Carregamento de credenciais
with open('/mnt/c/Users/renan.cunha/OneDrive - ST IT Tecnologia e Serviços Ltda/Documentos/Renan/Desafio Tech Del/project_langchain_langgraph_chat/code/cred.json', 'r') as f:
    credentials = json.load(f)
os.environ["TAVILY_API_KEY"] = credentials["tavily"]
os.environ["HUGGINGFACEHUB_API_TOKEN"] = credentials["huggingface"]

In [3]:
# Configuração do modelo
system_prompt = """Você é um assistente artificial capaz de conversar sobre diversos assuntos, 
                    exceto Engenharia Civil. Se o usuário mencionar esse tema, informe que não pode ajudar."""

hf_endpoint = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.5
)

/home/renancunha/miniconda3/envs/deloite_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm = ChatHuggingFace(llm=hf_endpoint)
search_tool = TavilySearchResults(max_results=10)
tools = [search_tool]
llm_with_tools = llm.bind_tools(tools)

graph_builder = StateGraph(dict)

In [5]:
def chatbot_node(state):
    messages = state["messages"]
    try:
        response = llm_with_tools.invoke(messages)
        print(f"Resposta do modelo: {response}")  # Debug
        return {"messages": messages + [response]}
    except Exception as e:
        print(f"Erro no modelo: {e}")
        return {"messages": messages + [{"role": "assistant", "content": f"Erro: {e}"}]}


In [6]:
graph_builder.add_node("chatbot", chatbot_node)

tool_node = ToolNode(tools)
graph_builder.add_node("tools", tool_node)

graph_builder.set_entry_point("chatbot")
graph_builder.add_edge("chatbot", END)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile()

In [7]:
def chat():
    messages = [{"role": "system", "content": system_prompt}]
    print("Assistente: Olá! Como posso ajudar você hoje? (Digite 'sair' para encerrar)")
    
    while True:
        user_input = input("Você: ")
        if user_input.lower() == 'sair':
            print("Assistente: Até logo!")
            break
        
        messages.append({"role": "user", "content": user_input})
        
        for event in graph.stream({"messages": messages}):
            print(f"Evento recebido: {event}")  # Debug
            if "chatbot" in event:
                response = event["chatbot"]["messages"][-1].content
                print(f"Resposta gerada: {response}")  # Debug
                print(f"Assistente: {response}")
                messages.append({"role": "assistant", "content": response})
                break

In [8]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "Qual é a capital do Brasil?"}
]

try:
    response = llm_with_tools.invoke(messages)
    print(f"Resposta direta do modelo: {response}")
except Exception as e:
    print(f"Erro ao invocar o modelo diretamente: {e}")


Resposta direta do modelo: content='' additional_kwargs={'tool_calls': [ChatCompletionOutputToolCall(function=ChatCompletionOutputFunctionDefinition(arguments={'query': 'qual é a capital do brasil?'}, name='tavily_search_results_json', description=None), id='0', type='function')]} response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=37, prompt_tokens=304, total_tokens=341), 'model': '', 'finish_reason': 'stop'} id='run-aa1c44f4-c665-4d4a-8f4d-ae7d5f04602a-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'qual é a capital do brasil?'}, 'id': '0', 'type': 'tool_call'}]


In [ ]:
if __name__ == "__main__":
    chat()

Assistente: Olá! Como posso ajudar você hoje? (Digite 'sair' para encerrar)
Assistente: 
Assistente: 
Assistente: 
Assistente: 
Assistente: 
Assistente: 
Assistente: 
